In [5]:
import tensorflow as tf
import numpy as np

tf.compat.v1.disable_eager_execution()
imported_graph = tf.compat.v1.train.import_meta_graph('./models/model-2000000.meta')

In [2]:
with tf.compat.v1.Session() as sess:
    # restore the saved vairable
    imported_graph.restore(sess, './models/model-2000000')
    #sess.predict()

INFO:tensorflow:Restoring parameters from ./models/model-2000000
